### Classification with additional features (Section 6.3)
* Zweck: Validierung von Klassifikation mit anderen Features als Includes und Function Calls.
* Matrix: Klassifikationsmatrix auf aktuellem Stand "current", sowie mit Feature History des mozilla-central Repository
* Features: Includes, Function Calls, Definitions, Names, Conditions
* Modell: Support Vector Machine Classifier

#### Setup
* Training-Set/Test-Set: Stratified sampling auf einer Matrix (2/3 : 1/3)

#### Results
Tabellarischer Vergleich der durchschnittlichen Precision und Recall Werte für verschiedene Features bei n=5 Experimenten. Weiter werden die Anzahl extrahierter Features und die durchschnittliche Laufzeit für das Trainieren des Modells aufgelistet.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from prettytable import PrettyTable

from imports.matrix_helper import MatrixHelper
from imports.prediction_helper import PredictionHelper
from sklearn.metrics import precision_recall_curve

matrix_helper = MatrixHelper()
experiments_per_feature = 5

features = [('incl', 'Includes'), ('cond', 'Conditions'), ('defs', 'Definitions'), ('names', 'Names'), ('calls', 'Function Calls')]
table = PrettyTable(['Features', 'Feature count', 'Precision', 'Recall', 'Precision sd', 'Recall sd', 'Time'])
table.align["Features"] = "l"

for feature in features:
    for h_type in ['current', 'history']:
        # Read pickle
        matrices = matrix_helper.load_from_parse('data/matrices/matrix_cla_{}_{}.pickle'.format(feature[0], h_type))
        
        feature_name = "{} ({})".format(feature[1], h_type)
        precision_list = []
        recall_list = []
        time_list = []
        
        for i in range(experiments_per_feature):
            print '* {:20}: {:2}/{:2}\r'.format(feature_name, i+1, experiments_per_feature),
            # Instantiate Prediction Helper Class and predict values for compare matrix with an SVM
            prediction_helper = PredictionHelper()
            prediction_helper.calculate_validation_compare_matrix(matrices, sampling_factor=(2.0/3), model_type='LinearSVC')
            compare_matrix = prediction_helper.get_compare_matrix()

            # Compute Precision-Recall
            precision, recall, thresholds = precision_recall_curve(np.array(compare_matrix[:, 2], dtype='f'), np.array(compare_matrix[:, 1], dtype='f'))
            precision_list.append(precision[1])
            recall_list.append(recall[1])
            time_list.append(prediction_helper.time_fitting)
        print
        
        divisor = float(experiments_per_feature)
        precision = "{:.3f}".format(sum(precision_list)/divisor)
        recall = "{:.3f}".format(sum(recall_list)/divisor)
        precision_sd = '{:.3f}'.format(np.std(precision_list))
        recall_sd = '{:.3f}'.format(np.std(recall_list))
        time = "{:.2f}min".format((sum(time_list)/divisor) / 60.0)
        
        table.add_row([feature_name, len(matrices[2]), precision, recall, precision_sd, recall_sd, time])

print(table)

* Includes (current)  :  5/ 5
* Includes (history)  :  5/ 5
* Conditions (current):  5/ 5
* Conditions (history):  5/ 5
* Definitions (current):  5/ 5
* Definitions (history):  5/ 5
* Names (current)     :  5/ 5
* Names (history)     :  5/ 5
* Function Calls (current):  5/ 5
* Function Calls (history):  5/ 5
+--------------------------+---------------+-----------+--------+--------------+-----------+---------+
| Features                 | Feature count | Precision | Recall | Precision sd | Recall sd |   Time  |
+--------------------------+---------------+-----------+--------+--------------+-----------+---------+
| Includes (current)       |     15362     |   0.688   | 0.328  |    0.022     |   0.018   | 0.04min |
| Includes (history)       |     16383     |   0.768   | 0.609  |    0.024     |   0.018   | 0.05min |
| Conditions (current)     |     19417     |   0.724   | 0.135  |    0.021     |   0.009   | 0.06min |
| Conditions (history)     |     19926     |   0.862   | 0.430  |    0.0